# A. Validación de datos VITALES Natalidad

## Carga de los datos

Este jupyter notebook esta generando los datos de un año en especifico (2022 el parquet part.0.parquet de vitales natalidad) para coincidir con las tablas compartidas del  power bi (Tablas agregadas) 

In [1]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("ValidaciónTablasVITALES_Natalidad").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/02 18:11:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [24]:
#spark.stop()

In [3]:
# Leer los archivos Parquet de un año 
df1_parquet = spark.read.parquet("/rawdata/vitales/vitales_parquet_2022/part.0.parquet")

df=df1_parquet

In [5]:
# Contar el número de registros en el DataFrame unido
total_registros = df.count()

# Listar las columnas del DataFrame unido
columnas = df.columns

# Mostrar resultados
print(f"Total de registros en el DataFrame unido: {total_registros}")
print(f"Columnas del DataFrame unido: {columnas}")

Total de registros en el DataFrame unido: 23632
Columnas del DataFrame unido: ['PersonaID', 'FechaNacimientoAAAAMM', 'Sexo', 'RegimenAfiliacionSGSSS', 'Administradora', 'CodigoPertenenciaEtnica', 'municipioResidencia', 'SemanasGestacion', 'EdadMadreMomentoDiagnostico', 'FechaCertificadoAAAAMM', 'IpsDondeRealizaParto', 'Peso', '__null_dask_index__']


In [6]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regIstros del año 2022 (128.343)
record_count = df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 23632


In [5]:
# Mostrar el esquema (columnas y tipos de datos)
df.printSchema()

# Listar las columnas
print(df.columns)

root
 |-- PersonaID: string (nullable = true)
 |-- FechaNacimientoAAAAMM: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- RegimenAfiliacionSGSSS: string (nullable = true)
 |-- Administradora: string (nullable = true)
 |-- CodigoPertenenciaEtnica: string (nullable = true)
 |-- municipioResidencia: string (nullable = true)
 |-- SemanasGestacion: string (nullable = true)
 |-- EdadMadreMomentoDiagnostico: string (nullable = true)
 |-- FechaCertificadoAAAAMM: string (nullable = true)
 |-- IpsDondeRealizaParto: string (nullable = true)
 |-- Peso: string (nullable = true)
 |-- __null_dask_index__: long (nullable = true)

['PersonaID', 'FechaNacimientoAAAAMM', 'Sexo', 'RegimenAfiliacionSGSSS', 'Administradora', 'CodigoPertenenciaEtnica', 'municipioResidencia', 'SemanasGestacion', 'EdadMadreMomentoDiagnostico', 'FechaCertificadoAAAAMM', 'IpsDondeRealizaParto', 'Peso', '__null_dask_index__']


# Empieza el proceso de generación de los datos agrupados por atenciones para natalidades

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
#spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df = df1

# Seleccionar solo las columnas deseadas (sin 'personaid_count' ya que no existe aún)
columns_to_keep = [
    'PersonaID','FechaNacimientoAAAAMM', 'RegimenAfiliacionSGSSS', 'Sexo','EdadMadreMomentoDiagnostico'
    
]
df_filtered = df.select(*columns_to_keep)

# Agregar una nueva columna con el recuento distintivo de 'personaid'
window_spec = Window.partitionBy('PersonaID')
df_filtered = df_filtered.withColumn('personaid_count', F.count('PersonaID').over(window_spec))

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered.show()

# Mostrar los nombres de las columnas filtradas para verificar
print("Nombres de las columnas filtradas:")
print(df_filtered.columns)

# Agrupar por la columna 'PersonaBasicaID'
grouped_df = df_filtered.groupBy('PersonaID').agg(
    F.first('FechaNacimientoAAAAMM').alias('FechaNacimientoAAAAMM'),
    F.first('RegimenAfiliacionSGSSS').alias('RegimenAfiliacionSGSSS'),
    F.first('Sexo').alias('Sexo'),
     F.first('EdadMadreMomentoDiagnostico').alias('EdadMadreMomentoDiagnostico'),
   
    
    F.first('personaid_count').alias('personaid_count')
)
# Mostrar las primeras filas del DataFrame agrupado para verificar
grouped_df.show()


+---------+---------------------+----------------------+---------+---------------------------+---------------+
|PersonaID|FechaNacimientoAAAAMM|RegimenAfiliacionSGSSS|     Sexo|EdadMadreMomentoDiagnostico|personaid_count|
+---------+---------------------+----------------------+---------+---------------------------+---------------+
|100000083|               202202|      C - CONTRIBUTIVO| FEMENINO|                         27|              1|
|100002313|               202204|        S - SUBSIDIADO| FEMENINO|                         28|              1|
|100004149|               202212|      C - CONTRIBUTIVO| FEMENINO|                         29|              1|
|100008250|               202205|        S - SUBSIDIADO|MASCULINO|                         30|              1|
|100011221|               202210|      C - CONTRIBUTIVO|MASCULINO|                         29|              1|
|100019351|               202203|        S - SUBSIDIADO|MASCULINO|                         33|              1|
|

+---------+---------------------+----------------------+---------+---------------------------+---------------+
|PersonaID|FechaNacimientoAAAAMM|RegimenAfiliacionSGSSS|     Sexo|EdadMadreMomentoDiagnostico|personaid_count|
+---------+---------------------+----------------------+---------+---------------------------+---------------+
|100000083|               202202|      C - CONTRIBUTIVO| FEMENINO|                         27|              1|
|100002313|               202204|        S - SUBSIDIADO| FEMENINO|                         28|              1|
|100004149|               202212|      C - CONTRIBUTIVO| FEMENINO|                         29|              1|
|100008250|               202205|        S - SUBSIDIADO|MASCULINO|                         30|              1|
|100011221|               202210|      C - CONTRIBUTIVO|MASCULINO|                         29|              1|
|100019351|               202203|        S - SUBSIDIADO|MASCULINO|                         33|              1|
|

In [18]:
# Contar el número de registros - me da satisfactoriamente la cantidad de registros en cada año ( 97.416)
record_count = grouped_df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 23621


Corresponde exactamente a la cantidad de registros agrupados

In [19]:
 df_final =grouped_df


In [20]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = df_final.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 23621


In [21]:
# Mostrar las primeras filas del DataFrame agrupado para verificar
df_final.show()

+---------+---------------------+----------------------+---------+---------------------------+---------------+
|PersonaID|FechaNacimientoAAAAMM|RegimenAfiliacionSGSSS|     Sexo|EdadMadreMomentoDiagnostico|personaid_count|
+---------+---------------------+----------------------+---------+---------------------------+---------------+
|100000083|               202202|      C - CONTRIBUTIVO| FEMENINO|                         27|              1|
|100002313|               202204|        S - SUBSIDIADO| FEMENINO|                         28|              1|
|100004149|               202212|      C - CONTRIBUTIVO| FEMENINO|                         29|              1|
|100008250|               202205|        S - SUBSIDIADO|MASCULINO|                         30|              1|
|100011221|               202210|      C - CONTRIBUTIVO|MASCULINO|                         29|              1|
|100019351|               202203|        S - SUBSIDIADO|MASCULINO|                         33|              1|
|

Se notifico que esta muestra tenia errores en la fase de exploración, no se han corregido

# Empieza el proceso de generación de tablas VITALES Natalidad

# Pagina 1 - Power BI

In [22]:
# Cargar el archivo Parquet en un DataFrame ( uno distinto por si deseo comparar)
df2 = df_final

## 1. Año, Mes, Sexo, EdadMadreMomentoDiagnostico, y RegimenAfiliacionSGSSS

In [23]:
# Selecciona la columna 'FechaRegistro' y muestra sus datos para comprobar como viene el formato fecha
#df2.select('FechaNacimientoAAAAMM').show(100, truncate=False)

+---------------------+
|FechaNacimientoAAAAMM|
+---------------------+
|202202               |
|202204               |
|202212               |
|202205               |
|202210               |
|202203               |
|202209               |
|202202               |
|202203               |
|202209               |
|202210               |
|202206               |
|202202               |
|202201               |
|202203               |
|202211               |
|202207               |
|202204               |
|202206               |
|202203               |
|202205               |
|202205               |
|202204               |
|202204               |
|202209               |
|202211               |
|202209               |
|202203               |
|202211               |
|202212               |
|202202               |
|202210               |
|202210               |
|202210               |
|202203               |
|202212               |
|202208               |
|202207               |
|202205         

In [24]:
from pyspark.sql import functions as F

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaID', 'FechaNacimientoAAAAMM', 'RegimenAfiliacionSGSSS', 'Sexo','EdadMadreMomentoDiagnostico', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaNacimientoAAAAMM' a año y mes correctamente
df_filtered = df_filtered.withColumn(
    'Año', F.substring(F.col('FechaNacimientoAAAAMM'), 1, 4).cast('int')
).withColumn(
    'Mes', F.substring(F.col('FechaNacimientoAAAAMM'), 5, 2).cast('int')
)

# Filtrar valores atípicos y nulos
df_filtered = df_filtered.filter(
    (F.col('Año').isNotNull()) & (F.col('Mes').isNotNull()) & (F.col('Año') > 1900)
)

# Agrupar por Año, Mes, Sexo, EdadMadreMomentoDiagnostico, y RegimenAfiliacionSGSSS
result_df = df_filtered.groupBy('Año', 'Mes', 'Sexo','EdadMadreMomentoDiagnostico', 'RegimenAfiliacionSGSSS').agg(
    F.countDistinct('PersonaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+---------+---------------------------+----------------------+------------------+--------------------+
| Año|Mes|     Sexo|EdadMadreMomentoDiagnostico|RegimenAfiliacionSGSSS|Número_de_Personas|Número_de_Atenciones|
+----+---+---------+---------------------------+----------------------+------------------+--------------------+
|2022|  1| FEMENINO|                         15|      C - CONTRIBUTIVO|                 2|                   2|
|2022|  8|MASCULINO|                         26|        S - SUBSIDIADO|                33|                  33|
|2022|  5|MASCULINO|                         28|      C - CONTRIBUTIVO|                19|                  19|
|2022| 10|MASCULINO|                         39|        S - SUBSIDIADO|                 9|                   9|
|2022|  1| FEMENINO|                         18|      C - CONTRIBUTIVO|                 9|                   9|
|2022|  5|MASCULINO|                         33|         E - EXCEPCION|                 1|              

Corresponde exactamente con los registros de powerBI

Año	Mes	Sexo	EdadMadreMomentoDiagnostico	RegimenAfiliacionSGSSS	# personas	# atenciones
2022	enero	FEMENINO	15	C - CONTRIBUTIVO	2	2

Año	Mes	Sexo	EdadMadreMomentoDiagnostico	RegimenAfiliacionSGSSS	# personas	# atenciones
2022	enero	FEMENINO	27	S - SUBSIDIADO	27	27

# B. Validación de tablas VITALES defunciones

In [25]:
# Leer los archivos Parquet de un año 
df1_parquet = spark.read.parquet("/rawdata/vitales/defunciones/defunciones_parquet_2008_a_2016/part.0.parquet")

df=df1_parquet


In [26]:
# Contar el número de registros en el DataFrame unido
total_registros = df.count()

# Listar las columnas del DataFrame unido
columnas = df.columns

# Mostrar resultados
print(f"Total de registros en el DataFrame unido: {total_registros}")
print(f"Columnas del DataFrame unido: {columnas}")

Total de registros en el DataFrame unido: 78647
Columnas del DataFrame unido: ['PersonaID', 'Sexo', 'RegimenAfiliacionSGSSS', 'Administradora', 'CodigoPertenenciaEtnica', 'municipioResidencia', 'SemanasGestacion', 'EdadMadreMomentoDiagnostico', 'FechaDefuncionAAAAMM', 'IpsDondeRealizaParto', 'Peso', 'AnoID']


In [27]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regIstros del año 2022 (128.343)
record_count = df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 78647


In [28]:
# Mostrar el esquema (columnas y tipos de datos)
df.printSchema()

# Listar las columnas
print(df.columns)

root
 |-- PersonaID: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- RegimenAfiliacionSGSSS: string (nullable = true)
 |-- Administradora: string (nullable = true)
 |-- CodigoPertenenciaEtnica: string (nullable = true)
 |-- municipioResidencia: string (nullable = true)
 |-- SemanasGestacion: string (nullable = true)
 |-- EdadMadreMomentoDiagnostico: string (nullable = true)
 |-- FechaDefuncionAAAAMM: string (nullable = true)
 |-- IpsDondeRealizaParto: string (nullable = true)
 |-- Peso: string (nullable = true)
 |-- AnoID: string (nullable = true)

['PersonaID', 'Sexo', 'RegimenAfiliacionSGSSS', 'Administradora', 'CodigoPertenenciaEtnica', 'municipioResidencia', 'SemanasGestacion', 'EdadMadreMomentoDiagnostico', 'FechaDefuncionAAAAMM', 'IpsDondeRealizaParto', 'Peso', 'AnoID']


# Empieza el proceso de generación de los datos agrupados por atenciones para defunciones

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
#spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df = df1

# Seleccionar solo las columnas deseadas (sin 'personaid_count' ya que no existe aún)
columns_to_keep = [
    'PersonaID','FechaDefuncionAAAAMM', 'RegimenAfiliacionSGSSS', 'Sexo','EdadMadreMomentoDiagnostico'
    
]
df_filtered = df.select(*columns_to_keep)

# Agregar una nueva columna con el recuento distintivo de 'personaid'
window_spec = Window.partitionBy('PersonaID')
df_filtered = df_filtered.withColumn('personaid_count', F.count('PersonaID').over(window_spec))

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered.show()

# Mostrar los nombres de las columnas filtradas para verificar
print("Nombres de las columnas filtradas:")
print(df_filtered.columns)

# Agrupar por la columna 'PersonaBasicaID'
grouped_df = df_filtered.groupBy('PersonaID').agg(
    F.first('FechaDefuncionAAAAMM').alias('FechaDefuncionAAAAMM'),
    F.first('RegimenAfiliacionSGSSS').alias('RegimenAfiliacionSGSSS'),
    F.first('Sexo').alias('Sexo'),
     F.first('EdadMadreMomentoDiagnostico').alias('EdadMadreMomentoDiagnostico'),
   
    
    F.first('personaid_count').alias('personaid_count')
)
# Mostrar las primeras filas del DataFrame agrupado para verificar
grouped_df.show()


+---------+--------------------+----------------------+-------------+---------------------------+---------------+
|PersonaID|FechaDefuncionAAAAMM|RegimenAfiliacionSGSSS|         Sexo|EdadMadreMomentoDiagnostico|personaid_count|
+---------+--------------------+----------------------+-------------+---------------------------+---------------+
|        1|              200801|      C - CONTRIBUTIVO|     FEMENINO|                         30|          54558|
|        1|              200903|        S - SUBSIDIADO|INDETERMINADO|                        198|          54558|
|        1|              200905|      C - CONTRIBUTIVO|     FEMENINO|                         68|          54558|
|        1|              201310|        S - SUBSIDIADO|    MASCULINO|                         78|          54558|
|        1|              201306|      C - CONTRIBUTIVO|    MASCULINO|                         85|          54558|
|        1|              201312|        S - SUBSIDIADO|INDETERMINADO|                   

In [35]:
# Contar el número de registros - me da satisfactoriamente la cantidad de registros en cada año ( 97.416)
record_count = grouped_df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 24090


In [36]:
 df_final =grouped_df

In [37]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = df_final.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 24090


In [38]:
# Mostrar las primeras filas del DataFrame agrupado para verificar
df_final.show()

+---------+--------------------+----------------------+---------+---------------------------+---------------+
|PersonaID|FechaDefuncionAAAAMM|RegimenAfiliacionSGSSS|     Sexo|EdadMadreMomentoDiagnostico|personaid_count|
+---------+--------------------+----------------------+---------+---------------------------+---------------+
|        1|              200801|      C - CONTRIBUTIVO| FEMENINO|                         30|          54558|
| 10000041|              201610|      C - CONTRIBUTIVO|MASCULINO|                         81|              1|
| 10000337|              201408|      C - CONTRIBUTIVO| FEMENINO|                         67|              1|
|100023165|              201604|        S - SUBSIDIADO| FEMENINO|                         23|              1|
|100024318|              201509|      C - CONTRIBUTIVO| FEMENINO|                          3|              1|
| 10003459|                  82|      C - CONTRIBUTIVO|MASCULINO|                          9|              1|
|100035367

En este caso la fuente de datos es practicamente la misma, solo cambia la variable fecha

# 1.   Año, Mes, Sexo, EdadMadreMomentoDiagnostico, y RegimenAfiliacionSGSSS

In [39]:
# Cargar el archivo Parquet en un DataFrame ( uno distinto por si deseo comparar)
df2 = df_final

In [40]:
from pyspark.sql import functions as F

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaID', 'FechaDefuncionAAAAMM', 'RegimenAfiliacionSGSSS', 'Sexo','EdadMadreMomentoDiagnostico', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaNacimientoAAAAMM' a año y mes correctamente
df_filtered = df_filtered.withColumn(
    'Año', F.substring(F.col('FechaDefuncionAAAAMM'), 1, 4).cast('int')
).withColumn(
    'Mes', F.substring(F.col('FechaDefuncionAAAAMM'), 5, 2).cast('int')
)

# Filtrar valores atípicos y nulos
df_filtered = df_filtered.filter(
    (F.col('Año').isNotNull()) & (F.col('Mes').isNotNull()) & (F.col('Año') > 1900)
)

# Agrupar por Año, Mes, Sexo, EdadMadreMomentoDiagnostico, y RegimenAfiliacionSGSSS
result_df = df_filtered.groupBy('Año', 'Mes', 'Sexo','EdadMadreMomentoDiagnostico', 'RegimenAfiliacionSGSSS').agg(
    F.countDistinct('PersonaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+---------+---------------------------+----------------------+------------------+--------------------+
| Año|Mes|     Sexo|EdadMadreMomentoDiagnostico|RegimenAfiliacionSGSSS|Número_de_Personas|Número_de_Atenciones|
+----+---+---------+---------------------------+----------------------+------------------+--------------------+
|2015| 10| FEMENINO|                         48|        S - SUBSIDIADO|                 1|                   1|
|2014|  7|MASCULINO|                         83|        S - SUBSIDIADO|                 3|                   3|
|2016|  9|MASCULINO|                         56|     NR - NO REPORTADO|                 2|                   2|
|2014|  6|MASCULINO|                         64|        S - SUBSIDIADO|                 2|                   2|
|2014| 10| FEMENINO|                         37|        S - SUBSIDIADO|                 1|                   1|
|2015| 11| FEMENINO|                         65|      C - CONTRIBUTIVO|                 2|              

In [41]:
spark.stop()